# 第七章 代理

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、LangChain内置工具](#二、LangChain内置工具)
     - [2.1 使用llm-math和wikipedia工具](#2.1-使用llm-math和wikipedia工具)
     - [2.2 使用PythonREPLTool工具](#2.2-使用PythonREPLTool工具)
        - [2.2.1 创建Python代理](#2.2.1-创建Python代理)
        - [2.2.2 使用调试模式](#2.2.2-使用调试模式)
 - [三、 定义自己的工具并在代理中使用](#三、-定义自己的工具并在代理中使用)
     - [3.1 创建和使用自定义时间工具](#3.1-创建和使用自定义时间工具)


## 一、设置OpenAI API Key

登陆 [OpenAI 账户](https://platform.openai.com/account/api-keys) 获取API Key，然后将其设置为环境变量。

- 如果你想要设置为全局环境变量，可以参考[知乎文章](https://zhuanlan.zhihu.com/p/627665725)。
- 如果你想要设置为本地/项目环境变量，在本文件目录下创建`.env`文件, 打开文件输入以下内容。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  替换"your_api_key"为你自己的 API Key

## 二、LangChain内置工具
### 2.1-使用llm-math和wikipedia工具

In [1]:
import os
from langchain_openai import ChatOpenAI #openai模型
from langchain.agents import load_tools, initialize_agent, AgentType, AgentExecutor, create_react_agent

API_KEY = os.environ.get("CHAT_ANYWHERE_API_KEY")
BASE_URL = "https://api.chatanywhere.com.cn/v1"

# 1、初始化大语言模型
llm = ChatOpenAI(base_url=BASE_URL, api_key=API_KEY, model="gpt-3.5-turbo", temperature=0.0)

# 2、加载工具包
# - `llm-math` 工具结合语言模型和计算器用以进行数学计算
# - `wikipedia`工具通过API连接到wikipedia进行搜索查询。
tools = load_tools(
    ["llm-math", "wikipedia"],
    llm=llm
)

# 3、初始化代理
# - `agent`: 代理类型。这里使用的是`AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION`。
#            其中`CHAT`代表代理模型为针对对话优化的模型，`REACT`代表针对REACT设计的提示模版。
# - `handle_parsing_errors`: 是否处理解析错误。当发生解析错误时，将错误信息返回给大模型，让其进行纠正。
# - `verbose`: 是否输出中间步骤结果。
agent: AgentExecutor = initialize_agent(
    tools, #第二步加载的工具
    llm, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)
agent

/Users/chubin.zheng/ENVS/llm-learning/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


AgentExecutor(verbose=True, tags=['chat-zero-shot-react-description'], agent=ChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the following questions as best you can. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Calculator, wikipedia\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a li

#### 1️⃣ 使用代理回答数学问题

In [2]:
agent.invoke("计算300的25%，思考过程请使用中文。")



> Entering new AgentExecutor chain...
Question: 计算300的25%，思考过程请使用中文。
Thought: 需要计算300的25%，可以通过计算300乘以25%来得到结果。
Action:
```
{
  "action": "Calculator",
  "action_input": "300*25%"
}
```

Observation: Answer: 75.0
Thought:Final Answer: 75.0

> Finished chain.


{'input': '计算300的25%，思考过程请使用中文。', 'output': '75.0'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
    
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我们需要计算300的25%，这个过程中需要用到乘法和除法。</p>

2. 模型基于思考采取行动（Action）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用计算器（calculator），输入300*0.25</p>
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观察</strong>：答案: 75.0</p>

4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: 我们的问题有了答案 </p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: 75.0 </p>
5. 以字典的形式给出最终答案。

#### 2️⃣ Tom M. Mitchell的书

In [5]:
question = "Tom M. Mitchell 是一位美国计算机科学家，并且是卡内基梅隆大学 (CMU) 的创始人大学教授。他写了什么书？思考过程请使用中文。"
agent.invoke(question)



> Entering new AgentExecutor chain...
Question: Tom M. Mitchell 是一位美国计算机科学家，并且是卡内基梅隆大学 (CMU) 的创始人大学教授。他写了什么书？
Thought: 我们可以使用维基百科来查找 Tom M. Mitchell 写过的书籍。
Action:
```
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

/Users/chubin.zheng/ENVS/llm-learning/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/chubin.zheng/ENVS/llm-learning/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

{'input': 'Tom M. Mitchell 是一位美国计算机科学家，并且是卡内基梅隆大学 (CMU) 的创始人大学教授。他写了什么书？思考过程请使用中文。',
 'output': 'Tom M. Mitchell 写了一本名为《Machine Learning》的书。'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我应该使用维基百科去搜索。</p>

2. 模型基于思考采取行动（Action）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用维基百科，输入Tom M. Mitchell</p>
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观测</strong>: 页面: Tom M. Mitchell，页面: Tom Mitchell (澳大利亚足球运动员)</p>

4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: Tom M. Mitchell写的书是Machine Learning </p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: Machine Learning </p>
5. 以字典的形式给出最终答案。


值得注意的是，模型每次运行推理的过程可能存在差异，但最终的结果一致。

### 2.2 使用PythonREPLTool工具
#### 2.2.1 创建Python代理

In [10]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool


# 创建pyhon代理
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True #输出中间步骤
)

# 使用代理对顾客名字进行排序
customer_list = ["小明","小黄","小红","小蓝","小橘","小绿",]

agent.invoke(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""")



> Entering new AgentExecutor chain...
我们可以使用Python库来将中文名字转换为拼音。
Action: Python_REPL
Action Input: from xpinyin import Pinyin
Observation: 
Thought:我们导入了xpinyin库，现在可以使用它来将中文名字转换为拼音。
Action: Python_REPL
Action Input: p = Pinyin()
Observation: 
Thought:我们创建了一个Pinyin对象，可以用它来转换中文名字为拼音。
Action: Python_REPL
Action Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']
Observation: 
Thought:我们有一个包含中文名字的列表。
Action: Python_REPL
Action Input: p.get_pinyin(names, '')
Observation: TypeError('ord() expected a character, but string of length 2 found')
Thought:我们需要将列表中的每个中文名字单独转换为拼音。
Action: Python_REPL
Action Input: p.get_pinyin('小明', '')
Observation: 
Thought:我们成功地将单个中文名字转换为拼音。
Action: Python_REPL
Action Input: pinyins = [p.get_pinyin(name, '') for name in names]
Observation: 
Thought:我们成功地将所有中文名字转换为拼音。
Final Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']

> Finished chain.


{'input': "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。",
 'output': "['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']"}

#### 2.2.2 使用调试模式

在调试（debug）模式下再次运行，我们可以把上面的6步分别对应到下面的具体流程
1. 模型对于接下来需要做什么，给出思考（Thought）
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor] Entering Chain run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] [12.25s] Exiting LLM run with output</p>
    - <p style="font-family:verdana; font-size:12px;color:green">[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [12.25s] Exiting Chain run with output</p>
2. 模型基于思考采取行动（Action), 因为使用的工具不同，Action的输出也和之前有所不同，这里输出的为python代码
    - <p style="font-family:verdana; font-size:12px;color:green"> [tool/start] [1:chain:AgentExecutor > 4:tool:Python REPL] Entering Tool run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [tool/end] [1:chain:AgentExecutor > 4:tool:Python REPL] [2.2239999999999998ms] Exiting Tool run with output</p>
3. 模型得到观察（Observation）   
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 5:chain:LLMChain] Entering Chain run with input</p>
4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）   
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] Entering LLM run with input</p>
    - <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [6.94s] Exiting LLM run with output</p>
    
5. 给出最终答案（Final Answer） 
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [6.94s] Exiting Chain run with output</p>
6. 返回最终答案。
    - <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor] [19.20s] Exiting Chain run with output</p>

In [11]:
import langchain
langchain.debug=True
agent.run(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""") 
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I don't

## 三、 定义自己的工具并在代理中使用

### 3.1 创建和使用自定义时间工具

In [16]:
# 导入tool函数装饰器
from langchain.agents import tool
from datetime import date


# 函数注释不写的话会报错：Function must have a docstring if description not provided.
# 代理会利用注释中的信息来调用这个函数（工具）
@tool()
def time(text: str) -> str:
    """
    返回今天的日期，用于任何需要知道今天日期的问题。\
    输入应该总是一个空字符串，\
    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。
    """
    return str(date.today())


agent = initialize_agent(
    tools=[time], #将刚刚创建的时间工具加入代理
    llm=llm, #初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)
agent.invoke("今天的日期是？思考过程请使用中文。")



> Entering new AgentExecutor chain...
Question: What is today's date? 

Thought: 我们可以使用工具来获取今天的日期。

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2024-06-16
Thought:Final Answer: 2024-06-16

> Finished chain.


{'input': '今天的日期是？思考过程请使用中文。', 'output': '2024-06-16'}

✅ **总结**

1. 模型对于接下来需要做什么，给出思考（Thought） 
   <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>：我需要使用 time 工具来获取今天的日期</p>

2. 模型基于思考采取行动（Action), 因为使用的工具不同，Action的输出也和之前有所不同，这里输出的为python代码
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>行动</strong>: 使用time工具，输入为空字符串</p>
    
   
3. 模型得到观察（Observation）
    <p style="font-family:verdana; font-size:12px;color:green"><strong>观测</strong>: 2023-07-04</p>
    
   
    
4. 基于观察，模型对于接下来需要做什么，给出思考（Thought）
    <p style="font-family:verdana; font-size:12px;color:green"> <strong>思考</strong>: 我已成功使用 time 工具检索到了今天的日期</p>

5. 给出最终答案（Final Answer）
     <p style="font-family:verdana; font-size:12px;color:green"> <strong>最终答案</strong>: 今天的日期是2023-07-04.</p>
6. 返回最终答案。